<center><a href="https://www.nvidia.cn/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>

<br>

# <font color="#76b900">**Notebook 5:** 处理大型文档</font>

<br>

在上一个 notebook 中，我们了解了运行状态链和知识库！最后，我们拥有了执行简单对话管理和自定义知识跟踪所需的所有工具。在此 notebook 中，我们将把类似的思路应用在大型文档上，看看这会使我们的 LLM 遇到些什么问题。

<br>

### **学习目标：**

* 熟悉文档加载器及其实用功能。
* 了解如何通过对文档进行分块来逐渐建立知识库，从而在有限的上下文空间中解析大型文档。
* 理解如何利用文档块进行渐进式重构（progressive recontextualization）、转换（coersion）和整合（consolidation），以及它们的局限。

<br>

### **思考问题：**

* 查看 ArxivParser 中的数据块，您会发现有些块没什么意义，或者在文本转换的过程中被损坏了。它是否会对这些数据块进行清理？
* 考虑一下文档摘要工作流（或其它需要处理大量文档块的工作流），应该多久执行一次，什么时候执行合理？

<br>

### **Notebook 版权声明：**

* 本 notebook 是 [**NVIDIA 深度学习培训中心**](https://www.nvidia.cn/training/)的课程[**《构建大语言模型 RAG 智能体》**](https://www.nvidia.cn/training/instructor-led-workshops/building-rag-agents-with-llms/)中的一部分，未经 NVIDIA 授权不得分发。

<br> 

### **环境设置：**


In [ ]:
## Necessary for Colab, not necessary for course environment
# %pip install -qq langchain langchain-nvidia-ai-endpoints gradio
# %pip install -qq arxiv pymupdf

# import os
# os.environ["NVIDIA_API_KEY"] = "nvapi-..."

from functools import partial
from rich.console import Console
from rich.style import Style
from rich.theme import Theme

console = Console()
base_style = Style(color="#76B900", bold=True)
pprint = partial(console.print, style=base_style)

In [ ]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
ChatNVIDIA.get_available_models()

In [ ]:
## Useful utility method for printing intermediate states
from langchain_core.runnables import RunnableLambda
from functools import partial

def RPrint(preface="State: "):
    def print_and_return(x, preface=""):
        print(f"{preface}{x}")
        return x
    return RunnableLambda(partial(print_and_return, preface=preface))

def PPrint(preface="State: "):
    def print_and_return(x, preface=""):
        pprint(preface, x)
        return x
    return RunnableLambda(partial(print_and_return, preface=preface))

----

<br>

## **第 1 部分：** 与文档聊天

此 notebook 将开始讨论用 LLM 与文档进行聊天。当前，在大型公共数据或特定数据上训练聊天模型的成本令人望而却步，而让 LLM 理解一系列 PDF 甚至 YouTube 视频的想法打开了更多可能性！

* **您的 LLM 可以拥有一个基于人类可读文档构建的可修改知识库，**这意味着您可以直接控制它能访问什么样的数据，并指示它与其交互。
* **您的 LLM 可以直接从文档集中整理和提取引用信息。**借助充分的提示工程和指令遵循先验，您可以强制模型仅根据您提供的材料执行动作。
* **您的 LLM 甚至可以与您的文档进行交互，必要的时候执行自动修改。**这为自动内容优化和合成开辟了道路，稍后将深入探讨。

您可以尽情发挥想象力，想象更多的应用场景。下面就来看看如何实现这些！

<br>

#### **朴素的方法：将文档塞给模型**

假设您有一些文本文档（PDF、博客等），并想要对文档相关内容进行提问。其中一个可以尝试的方法就是把文档的某种表示一股脑塞进聊天模型！从文档的角度来看，这称为[**文档填充**（document stuffing）](https://python.langchain.com/docs/modules/chains/document/stuff)。

> <img src="https://dli-lms.s3.amazonaws.com/assets/s-fx-15-v1/imgs/doc_stuff.png" width=800px/>
>
> From [**Stuff | LangChain**🦜️🔗](https://python.langchain.com/docs/modules/chains/document/stuff)

<br>

如果您的模型足够强大且文档足够短，这个方法可能会有不错的效果，但不应该期待它会在整个文档中都表现的很好。由于训练限制，许多现代 LLM 在处理长上下文时都会有很大的问题。虽然无论您使用当今的哪种大模型，退化并不会带来灾难性后果，但它可能很快就无法正常地遵循指令了。
  
<br>

**文档推理需要解决的关键问题是：**

* 如何将文档分割成可推理的部分？
* 随着文档大小和数量的增加，我们如何高效地查找和考虑这些文档块？

本课程将探索解决这些问题的几种方法，同时继续培养 LLM 编排技能。**本 notebook 将有助于扩展我们之前的运行链技能，实现渐进式的推理，而下一个 notebook 将介绍大规模检索的新技术。**我们将继续利用先进的开源工具，来构建标准、可集成的解决方案。

文档加载框架有很多选择，整个课程中将涉及两个主要的框架：
* [**LangChain**](https://python.langchain.com/docs/get_started/introduction) 提供了一个简单的框架，可以通过常规分块策略将 LLM 连接到您自己的数据源，并与嵌入（embedding）框架/服务一同工作。此框架最初是为支持 LLM 而开发，其优势在于链的抽象和协调智能体。
* [**LlamaIndex**](https://gpt-index.readthedocs.io/en/stable/) 是一个数据框架，供 LLM 应用提取、构建和访问私有或领域特定数据。后来也扩展成类似 LangChain 的通用 LLM 框架，但目前它最擅长的仍然是为 LLM 处理文档，毕竟它最初就是为此设计的。

十分推荐您详细了解一下 LlamaIndex 和 LangChain 各自的优势，选择最适合您的。由于 LlamaIndex 可与 LangChain *一起*使用，因此两个框架的功能[可以结合使用](https://docs.llamaindex.ai/en/stable/community/integrations/using_with_langchain.html)。为简单起见，我们将在本课程中一直用 LangChain，对 LlamaIndex 感兴趣的学员可以通过 [**NVIDIA/GenerativeAIExamples 代码库**](https://github.com/NVIDIA/GenerativeAIExamples/tree/main/RetrievalAugmentedGeneration/notebooks)进一步探索。

----

<br>

## **第 2 部分：** 加载文档

LangChain 提供了各种[文档加载器](https://python.langchain.com/docs/integrations/document_loaders)以从不同的来源和位置（本地存储、私有 s3 存储桶、公共网站、消息 API 等）读取各种文档（HTML、PDF、代码）。这些加载程序会查询您的数据源并返回包含内容和元数据的 `Document` 对象，通常是纯文本或人类可读的格式。有许多可用的文档加载程序，[这里](https://python.langchain.com/docs/integrations/document_loaders)列出了 LangChain 的几个第一方选项。

**在本示例中，我们可以使用以下 LangChain 的加载器之一加载研究论文：**
* [`UnstructuredFileLoader`](https://python.langchain.com/docs/integrations/document_loaders/unstructured_file)：适用于任意文件的文档加载器，不会对文档结构做太多假设，通常够用了。
* [`ArxivLoader`](https://python.langchain.com/docs/integrations/document_loaders/arxiv)：一个更专业的文件加载程序，可以直接与 Arxiv 接口通信。[仅举一个例子](https://python.langchain.com/docs/integrations/document_loaders)，这将对数据做出更多假设，以生成更好的解析并自动填充元数据（当您有多种文档/格式时很有用）。

我们的代码示例会默认使用 `ArxivLoader` 来加载 [MRKL](https://arxiv.org/abs/2205.00445) 或 [ReAct](https://arxiv.org/abs/2210.03629) 文章，您很有可能在继续研究聊天模型的某个阶段读到这些论文。

In [ ]:
%%time
from langchain.document_loaders import UnstructuredFileLoader
from langchain.document_loaders import ArxivLoader

## Loading in the file

## Unstructured File Loader: Good for arbitrary "probably good enough" loader
# documents = UnstructuredFileLoader("llama2_paper.pdf").load()

## More specialized loader, won't work for everything, but simple API and usually better results
documents = ArxivLoader(query="2404.16130").load()  ## GraphRAG
# documents = ArxivLoader(query="2404.03622").load()  ## Visualization-of-Thought
# documents = ArxivLoader(query="2404.19756").load()  ## KAN: Kolmogorov-Arnold Networks
# documents = ArxivLoader(query="2404.07143").load()  ## Infini-Attention
# documents = ArxivLoader(query="2210.03629").load()  ## ReAct

<br>

从导入中我们可以看到，此连接器（connector）允许我们访问两个不同的组件：
* `page_content` 实际上就是人类可读格式的文档正文。
* `metadata` 是连接器通过其数据源提供的文档相关信息。

下面我们就来看看文档正文中都包括什么，您可能会注意到文档的长度有点不受控制了：

In [ ]:
## Printing out a sample of the content
print("Number of Documents Retrieved:", len(documents))
print(f"Sample of Document 1 Content (Total Length: {len(documents[0].page_content)}):")
print(documents[0].page_content[:1000])

<br>  

相比之下，元数据的大小更精简，足够作为您的聊天模型的上下文组件了：

In [ ]:
pprint(documents[0].metadata)

<br>  

尽管按原样保留元数据格式并完全忽略正文听起来好像很有吸引力，但如果不深入完整文本，就无法获取一些关键特征：
* **不一定保证有元数据。**对于 `arxiv`，论文摘要、标题、作者和日期是提交文章所必须包含的信息，因此能查到它们并不稀奇。但对于任意的一个 PDF 或网页，情况就不一定了。
* **智能体将无法更深入地了解文档内容。**摘要方便我们了解信息，可以按原样保留，但它并不提供与正文进行任意交互的能力。
* **智能体仍然无法同时推理太多文档。**也许在 MRKL/ReAct 示例中，您可以将这两个摘要合并到一个上下文中提问。那当您需要同时与 5 个文档进行交互时怎么办呢？整个目录呢？您很快就会发现，哪怕只是总结和列出这些文档，都会使上下文窗口信息超载！

----

<br>

## **第 3 部分** 转换文档

文档加载完成后，要是我们想把它作为上下文传给 LLM，通常要先做一步转换。一种转换方法是**分块**（chunking），它把大段的内容分解成小段。这种技巧很有用，因为它能[优化从向量数据库返回内容的相关性](https://www.pinecone.io/learn/chunking-strategies/)。

LangChain 提供了[各种文档转换器](https://python.langchain.com/docs/integrations/document_transformers/)，我们将使用 [`RecursiveCharacterTextSplitter`](https://python.langchain.com/docs/modules/data_connection/document_transformers/text_splitters/recursive_text_splitter)。它将帮我们遵循一些自然的停止点来分割文档（越多越好）。

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1200,
    chunk_overlap=100,
    separators=["\n\n", "\n", ".", ";", ",", " ", ""],
)

## Some nice custom preprocessing
# documents[0].page_content = documents[0].page_content.replace(". .", "")
docs_split = text_splitter.split_documents(documents)

# def include_doc(doc):
#     ## Some chunks will be overburdened with useless numerical data, so we'll filter it out
#     string = doc.page_content
#     if len([l for l in string if l.isalpha()]) < (len(string)//2):
#         return False
#     return True

# docs_split = [doc for doc in docs_split if include_doc(doc)]
print(len(docs_split))

In [ ]:
for i in (0, 1, 2, 15, -1):
    pprint(f"[Document {i}]")
    print(docs_split[i].page_content)
    pprint("="*64)

<br>  

我们现在用的的分块方法很朴素，但至少能看出来我们可以轻松地让应用跑起来。我们为得到较小的分块做了一些努力，以便模型能有效地将其作为上下文，但我们要怎么把这些块用于推理呢？

**在针对一组任意文档扩展和优化此方法时，可以借鉴以下做法：**

* 识别逻辑中断的技术，以及合成文本的技术（手动、自动、或者 LLM 辅助）。
* 构建信息丰富且独特的数据块，避免冗余，从而最大限度提高数据库效用。
* 自定义分块以适应文档的特性，确保分块与上下文相关且一致。
* 在每个数据块中都放入关键概念、关键词或元数据片段，以提高数据库的可搜索性和相关性。
* 持续评估分块效果，并随时准备调整策略，以在分块大小和内容丰富度间取得最佳平衡。
* 考虑使用层级结构（隐式生成或明确指定）来改进检索。
	+ 如果您有兴趣，可以查看[索引指南中的 LlamaIndex 树结构](https://docs.llamaindex.ai/en/stable/module_guides/indexing/index_guide.html#tree-index)。

----

<br>

## **第 4 部分：[练习]** 重构摘要（Refining Summaries）

为了自动推理大量文档，一个可能的想法是用 LLM 创建一个压缩的摘要或知识库。与我们在上一个 notebook 中通过填充槽位来维护对话历史记录类似，保持整个文档的历史记录会有什么问题么？

本节我们将重点介绍一个令人兴奋的 LLM 应用：**大规模自动重构（refine）、转换（coerce）和整合（consolidate）数据**。具体来说，我们将实现一个简单但实用的运行时，用 while 循环和运行状态链机制来总结一系列文档块。这个过程通常被称为[**“文档重构”**](https://python.langchain.com/docs/modules/chains/document/refine)，很大程度上类似于我们之前的对话填槽练习。唯一的区别是我们现在处理的是大型文档，而不是进行中的聊天记录。

> <img src="https://dli-lms.s3.amazonaws.com/assets/s-fx-15-v1/imgs/doc_refine.png" width=1000px/>
>
> From [**Refine | LangChain**🦜️🔗](https://python.langchain.com/docs/modules/chains/document/refine)

<br>

#### **DocumentSummaryBase 模型**

就像上一个 notebook 的 `KnowledgeBase` 类，我们可以创建一个 `DocumentSummaryBase` 结构来封装文档。下面的例子就将用 `running_summary` 字段来查询模型以获得最终摘要，同时用 `main_ideas` 和 `loose_ends` 字段来防止摘要演进过快。我们需要通过提示工程来保证这一点，因此用到了 `summary_prompt`。您可以根据需要对其进行修改，以适用于您的模型。

In [ ]:
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables.passthrough import RunnableAssign
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.output_parsers import PydanticOutputParser

from langchain_nvidia_ai_endpoints import ChatNVIDIA

from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List
from IPython.display import clear_output


class DocumentSummaryBase(BaseModel):
    running_summary: str = Field("", description="Running description of the document. Do not override; only update!")
    main_ideas: List[str] = Field([], description="Most important information from the document (max 3)")
    loose_ends: List[str] = Field([], description="Open questions that would be good to incorporate into summary, but that are yet unknown (max 3)")


summary_prompt = ChatPromptTemplate.from_template(
    "You are generating a running summary of the document. Make it readable by a technical user."
    " After this, the old knowledge base will be replaced by the new one. Make sure a reader can still understand everything."
    " Keep it short, but as dense and useful as possible! The information should flow from chunk to (loose ends or main ideas) to running_summary."
    " The updated knowledge base keep all of the information from running_summary here: {info_base}."
    "\n\n{format_instructions}. Follow the format precisely, including quotations and commas"
    "\n\nWithout losing any of the info, update the knowledge base with the following: {input}"
)

<br>  

现在正好可以用上前一个 notebook 的 `RExtract` 函数：

In [ ]:
def RExtract(pydantic_class, llm, prompt):
    '''
    Runnable Extraction module
    Returns a knowledge dictionary populated by slot-filling extraction
    '''
    parser = PydanticOutputParser(pydantic_object=pydantic_class)
    instruct_merge = RunnableAssign({'format_instructions' : lambda x: parser.get_format_instructions()})
    def preparse(string):
        if '{' not in string: string = '{' + string
        if '}' not in string: string = string + '}'
        string = (string
            .replace("\\_", "_")
            .replace("\n", " ")
            .replace("\]", "]")
            .replace("\[", "[")
        )
        # print(string)  ## Good for diagnostics
        return string
    return instruct_merge | prompt | llm | preparse | parser


<br>

记住，以下代码会在 for 循环中调用正在运行的状态链来迭代文档！唯一需要实现的是 `parse_chain`，它需要能够将状态正确地传递到 `RExtract`。之后，系统应能正常工作，动态地维护文档摘要（针对使用的模型，可能需要调整提示词）。

In [ ]:
latest_summary = ""

## TODO: Use the techniques from the previous notebook to complete the exercise
def RSummarizer(knowledge, llm, prompt, verbose=False):
    '''
    Exercise: Create a chain that summarizes
    '''
    ###########################################################################################
    ## START TODO:

    def summarize_docs(docs):        
        ## TODO: Initialize the parse_chain appropriately; should include an RExtract instance.
        ## HINT: You can get a class using the <object>.__class__ attribute...
        parse_chain = RunnableAssign({'info_base' : (lambda x: None)})
        ## TODO: Initialize a valid starting state. Should be similar to notebook 4
        state = {}

        global latest_summary  ## If your loop crashes, you can check out the latest_summary
        
        for i, doc in enumerate(docs):
            ## TODO: Update the state as appropriate using your parse_chain component

            assert 'info_base' in state 
            if verbose:
                print(f"Considered {i+1} documents")
                pprint(state['info_base'])
                latest_summary = state['info_base']
                clear_output(wait=True)

        return state['info_base']
        
    ## END TODO
    ###########################################################################################
    
    return RunnableLambda(summarize_docs)

# instruct_model = ChatNVIDIA(model="mistralai/mixtral-8x7b-instruct-v0.1").bind(max_tokens=4096)
instruct_model = ChatNVIDIA(model="mistralai/mixtral-8x22b-instruct-v0.1").bind(max_tokens=4096)
instruct_llm = instruct_model | StrOutputParser()

## Take the first 10 document chunks and accumulate a DocumentSummaryBase
summarizer = RSummarizer(DocumentSummaryBase(), instruct_llm, summary_prompt, verbose=True)
summary = summarizer.invoke(docs_split[:15])

In [ ]:
pprint(latest_summary)

----

<br>

## **第 5 部分：** 处理合成数据

在结束对使用 LLM 做文档摘要的探索之前，值得花点时间探讨一下更广的场景和潜在挑战。我们已经展示了一种能提取出简洁、有意义摘要的方法，现在考虑一下为什么这是个至关重要的方法，以及其中涉及的复杂性。

#### **泛化重构**

需要注意，这种“渐进式总结”技术只是一种入门级链，对初始数据和所需的输出格式几乎不做假设。相同的技术可以泛化到用来生成包含已知元数据、存在大量假设和下游目标的合成内容上。

**考虑以下潜在应用：**

1. **聚合数据**（Aggregating Data）：构建一种结构，将原始数据从文档块转换为一致、有用的摘要。
2. **分类和子主题分析**：创建一个系统，将洞察从数据块提炼到预定的类别中，并跟踪其中新出现的子主题。
3. **整合为密集信息块**：优化这些结构，将洞察提炼为紧凑的段，通过直接引用进行更深入的分析。

这些应用都预示需要创建一个聊天模型能访问的**特定领域知识图谱**。已经有一些应用可以自动化地生成图谱了，比如 [**LangChain 知识图谱**](https://python.langchain.com/docs/modules/memory/types/kg)。虽然您可能需要开发层次化的（hierachical）结构和工具来构建和遍历这种结构，但如果能为您的场景构建出一个好用的知识图谱，那一点都不亏！对那些有兴趣构建高级知识图谱（依赖更庞大系统和向量相似性）的学员，我们发现 [**LangChain x Neo4j 文章**](https://blog.langchain.dev/using-a-knowledge-graph-to-implement-a-devops-rag-application/) 很有帮助。

### **大规模数据处理的挑战**

虽然我们的方法让人很有想象空间，但它仍然在处理大量数据时面临挑战：

* **预处理的局限**：尽管做摘要是相对简单的，但开发在各种场景都普遍有效的层次结构是很有挑战的。
* **粒度和引导（navigation）成本**：在层次结构中实现精细的粒度可能需要大量资源，需要复杂的整合或很多分支来保持每次交互的可管理上下文大小。
* **对精确指令执行的依赖**：用我们当前的工具引导层次结构的检索，很依赖在强大的指令微调模型上做大量提示工程。推理延迟和参数预测中的错误风险可能会很大，因此用 LLM 实现会成为一个挑战。

在您继续学习课程的过程中，想想这些挑战是靠什么技术解决的。

-----

<br>

## **第 6 部分：** 总结

此 notebook 的目标是介绍聊天模型如何处理大型文档。下一个 notebook，我们将研究一个有不同优劣势的补充工具：**使用嵌入模型进行语义检索**。

### <font color="#76b900">**非常好！**</font>

### **接下来：**
1. **[可选]** 回顾 notebook 顶部的“思考问题”。
2. **[可选]** 此 notebook 包含一些基本的文档处理链，但不涉及 [Map Reduce](https://python.langchain.com/docs/modules/chains/document/map_reduce) 和 [Map Rerank](https://python.langchain.com/docs/modules/chains/document/map_rerank) 链，它们也非常有用，而且是基于大致相同的直觉构建的。了解一下有助于您加深理解！